In [39]:
!pip install openai
import openai
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import time
from tqdm import tqdm
import numpy as np

openai.api_key = "_"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def get_embedding(text , model = "text-embedding-ada-002"):
   # get the embedding of a sentence from ada
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model  )['data'][0]['embedding']

In [42]:
embs = []
quran = pd.read_excel("/content/Dataset-Verse-by-Verse.xlsx")
for i, verse in tqdm(enumerate(quran['ArabicText'])):
  try :
    embs.append((verse , get_embedding(verse)))
  except :
    # limit reached , so wait a minute then try again
    time.sleep(60)
    embs.append((verse , get_embedding(verse)))

6236it [1:44:02,  1.00s/it]


In [63]:
chat_gpt_embs = pd.DataFrame(data = list(zip([e[0] for e in embs] , [e[1] for e in embs])))
chat_gpt_embs = chat_gpt_embs.drop_duplicates(subset=0)

In [198]:
def get_emb_for_str(text):
    return np.array(chat_gpt_embs[chat_gpt_embs[0]==text][1].item()).reshape((1,-1))

def pair_wise_similarity(text1,text2):
  emb1= get_emb_for_str(text1)
  emb2=get_emb_for_str(text2)
  return cosine_similarity(emb1 ,emb2).item()

In [118]:
pairs_dataset =pd.read_csv("pairs_quran_dataset.csv")

In [146]:
L=[]
for i in range(7679):
    L.append(pair_wise_similarity(pairs_dataset.iloc[i]['premise'], pairs_dataset.iloc[i]['hypothesis']))

In [147]:
similarity_dataset = pairs_dataset

In [148]:
similarity_dataset['sim_scores'] = pd.Series(L)

In [150]:
similarity_dataset[ 'sim_scores' ] = similarity_dataset['sim_scores'].fillna(int(0))

In [151]:
TN = (similarity_dataset[similarity_dataset['label'] == 0]['sim_scores'] < 0.6).sum()

In [152]:
TP = (similarity_dataset[similarity_dataset['label'] == 2]['sim_scores'] > 0.6).sum() + (similarity_dataset[similarity_dataset['label'] == 1]['sim_scores'] > 0.6).sum()
FN = (similarity_dataset[similarity_dataset['label'] == 2]['sim_scores'] < 0.6).sum() + (similarity_dataset[similarity_dataset['label'] == 1]['sim_scores'] < 0.6).sum()
FP = (similarity_dataset[similarity_dataset['label'] == 0]['sim_scores'] > 0.6).sum()

In [153]:
# accuracy
print('Accuracy : ' , (TP + TN) / (TP + TN + FP + FN))
# precision
print('Precision : ' , TP / (TP + FP))
# recall
print('Recall : ', TP / (TP + FN))
# F1 score

print('F1 score : ' , (2 * TP) / (2 * TP + FP + FN))

Accuracy :  0.8851412944393802
Precision :  0.8851412944393802
Recall :  1.0
F1 score :  0.9390715667311412


In [ ]:
!pip install sentence-transformers
from sentence_transformers.util import cos_sim

In [173]:
all_embs = np.array([np.array(x) for x in chat_gpt_embs[1].tolist()])

In [183]:
chat_gpt_embs = chat_gpt_embs.reset_index(drop=True)

In [199]:
sentence1 = "الحمد لله رب العالمين"
emb1 = get_emb_for_str(sentence1)
similarity_scores = cos_sim(emb1 , all_embs)[0].numpy().tolist()
top_10_indecies = np.argsort(similarity_scores)[::-1][:11]
results = list(zip(chat_gpt_embs[0][top_10_indecies].tolist() , sorted(similarity_scores)[::-1][:11]))
results

[('الحمد لله رب العالمين', 1.0000000000000002),
 ('والحمد لله رب العالمين', 0.9883528744137107),
 ('فلله الحمد رب السماوات ورب الأرض رب العالمين', 0.9413920880329145),
 ('هو الحي لا إله إلا هو فادعوه مخلصين له الدين ۗ الحمد لله رب العالمين',
  0.9016505910838588),
 ('قال فرعون وما رب العالمين', 0.896234765849896),
 ('بسم الله الرحمن الرحيم', 0.8909709285314888),
 ('الله ربكم ورب آبائكم الأولين', 0.8896969913893943),
 ('تنزيل من رب العالمين', 0.8864674340990158),
 ('هدًى ورحمةً للمحسنين', 0.886195638479871),
 ('ولولا فضل الله عليكم ورحمته وأن الله رءوف رحيم', 0.8848699403107538),
 ('فقطع دابر القوم الذين ظلموا ۚ والحمد لله رب العالمين', 0.8841729886166388)]

In [200]:
chat_gpt_embs.to_csv("gpt_embeddings.csv" , index=False)

In [201]:
def get_embedding(text , model = "text-embedding-ada-002"):
   # get the embedding of a sentence from ada
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model  )['data'][0]['embedding']
x = get_embedding("الصبر")

In [204]:
similarity_scores = cos_sim(np.array(x).reshape((1,-1)) , all_embs)[0].numpy().tolist()
top_10_indecies = np.argsort(similarity_scores)[::-1][:11]
results = list(zip(chat_gpt_embs[0][top_10_indecies].tolist() , sorted(similarity_scores)[::-1][:11]))
results

[('فاصبر صبرًا جميلًا', 0.8960826832144323),
 ('ولمن صبر وغفر إن ذلك لمن عزم الأمور', 0.8794731518459877),
 ('ولربك فاصبر', 0.8729079733916064),
 ('الرحمن', 0.8701296296529907),
 ('واستعينوا بالصبر والصلاة ۚ وإنها لكبيرة إلا على الخاشعين',
  0.8692332646264371),
 ('إلا الذين آمنوا وعملوا الصالحات وتواصوا بالحق وتواصوا بالصبر',
  0.8656808733374061),
 ('الذين صبروا وعلى ربهم يتوكلون', 0.8650750487787456),
 ('وجزاهم بما صبروا جنةً وحريرًا', 0.864442710527497),
 ('واصبر وما صبرك إلا بالله ۚ ولا تحزن عليهم ولا تك في ضيقٍ مما يمكرون',
  0.8641887351196579),
 ('ثم كان من الذين آمنوا وتواصوا بالصبر وتواصوا بالمرحمة', 0.8632413423551943),
 ('يا أيها الذين آمنوا استعينوا بالصبر والصلاة ۚ إن الله مع الصابرين',
  0.8612517193903815)]

In [211]:
def general_inference(prompt):
  x = get_embedding(prompt)
  similarity_scores = cos_sim(np.array(x).reshape((1,-1)) , all_embs)[0].numpy().tolist()
  top_10_indecies = np.argsort(similarity_scores)[::-1][:11]
  results = list(zip(chat_gpt_embs[0][top_10_indecies].tolist() ,  sorted(similarity_scores)[::-1][:11]  ))
  return results

In [213]:
def verse_inference(verse_prompt):
  emb1 = get_emb_for_str(verse_prompt)
  similarity_scores = cos_sim(emb1 , all_embs)[0].numpy().tolist()
  top_10_indecies = np.argsort(similarity_scores)[::-1][:11]
  results = list(zip(chat_gpt_embs[0][top_10_indecies].tolist() , sorted(similarity_scores)[::-1][:11]))
  return results

In [227]:
def similarity_inference(prompt):
  try : 
    return verse_inference(prompt)
  except :
    print("Using ada's api!")
    return general_inference(prompt) 

In [ ]:
similarity_inference("الزواج")